# 构建模型

一旦数据收集标注完毕,衡量指标选择完成.接下来要进行模型的开发了.

许多资料都认为构建模型是机器学习项目唯一的步骤而跳过了问题定义和数据收集.认为这两个过程已经由其他人完成.但是显然这里作者表达了不同意见.构建模型只是机器学习项目的一环,而且还不是最难的一环.(最难的是创建数据集).


## 准备数据

通过前面的例子,我们应该清楚,模型处理的数据通常并不是原始数据.我们需要进行预处理,将原始数据转换成更适合神经网络的格式.这个过程有

- 张量化(矢量化)
- 规范化
- 处理缺失值

其实我们在前面的例子中已经实践过这些步骤了,这里理论上回顾一下.


### 张量化

神经网络所有的数据都是典型的带浮点数的张量,任何输入神经网络的数据都要转换成张量输入.

imdb 数据集的例子中,我们将词频前 10000 的单词序列转换成了 32 位浮点数的张量.

当然如果数据已经是张量了,这个步骤可用跳过.


### 规范化

mnist 的例子中,每个像素是 0-255 的整数,我们将其转换成了 float32 / 255.最终范围是 0-1 的浮点数.

波士顿房价预测例子中,有非常多的特征,这些特征的类型不同,有整数有浮点数,相同类型的取值范围相差又相当大.最终我们是将所有特征都进行了规范化,使其标准差为 1,均值为 0.

一般而言,取值较大的数据或者内部差异很大的数据(两个特征,一个取值范围是0-1,另一个是 100-200)输入神经网络并不安全.这样会引发大的梯度更新,阻碍网络收敛.

适合输入神经网络的数据特征

- 取值范围较小,通常是 0-1.
- 同质化,不同特征的取值范围大致相同.

在此之上也可以更加严格的同一化,每个特征单独粗规范化.

- 均值为 0.
- 标准差为 1.

更加严格的同一化并不是必须的,需要按照问题分析.(mnist 显然没必要)

严格均一化显然 numpy 非常适合.

```py
x -= x.mean(axis=0)
x /= x.std(axis=0)
```


### 处理缺失值

实际收集的样本中,某些类型数据缺失是常有的事情.波士顿房价预测中人均犯罪率就不是所有样本都存在.

缺失值的样本超多,那可以选择放弃这个特征.但是还有其他处理方式.

- 如果这个特征是分类,那么数值缺失也可以算是一种类别.增加一个新的类别-数据缺失.
- 如果特征是数字类型,要避免将数据缺失填充成一个任意的值(例如 0 1),这样的任意值可能会在特征潜在的流行空间产生不连续,阻碍模型拟合.可以选择使用这个特征的平均值或中位数填充.或者相反可以训练另一个模型,使用样本的其他特征来预测缺失值.

还有一种情况是测试集存在缺失值,但是模型是在完整的训练集训练的.这种情况下,模型不会忽略缺失值.这种情况下需要人为的产生带缺失值的训练样本.

- 将一些训练样本复制若干次.
- 在一些批次放弃测试集中可能出现缺失值的特征.


## 选择验证指标(协议?原文是 protocol)

机器学习的最终目的是模型泛化能力 MAX,而任何对模型参数的调整都需要有准确的依据,而给这个依据的就是验证协议.

第五章我们提到了 3 种验证协议

- 简单留出验证,当有大量数据时
- K 折验证,当样本数量不足
- 重复 K 折验证,当样本数量不足,又需要对模型的高度精确评价.

任取其一即可,大部分情况下,简单留出验证足矣.


## 击败基线

当工作进展到模型本身时,第一个任务是开发一个能战胜基线的小模型.(感觉类似雏形).

这个阶段,要特别关注下面 3 个事情

- 特征工程: 过滤掉不具备参考价值的信息,使用已有的知识和对问题的理解,发掘新的特征.
- 选择正确的网络架构: 全连接网络? 卷积神经网络? 循环神经网络(RNN)? Transformer? 甚至要考虑这个问题使用深度学习求解是不是合适?是不是存在其他更好的解法?
- 足够好的训练配置: 损失函数? 训练批量大小? 学习率?

对于大部分问题,其实都有模板.我们是第一个应用深度学习的人,可能性太小了.确保自己已经查找过相关研究.找出前人是如何解决问题的,他们使用了哪些特征?又选择了怎样的网络架构?

while 世事难料,假如我们无论如何尝试合理的架构,还是没能完成一个打败基线的模型时,最可能的原因是输入数据没有包含问题的答案.

当我们进展到构建模型时,对手上的数据有两个假设

- 问题的答案可以从现有数据得出.
- 现有数据存在足够的信息得出答案.

如果无法打败基线,这两个假设可能是错误的,此时需要回到数据或问题,检查假设失败原因.


### 选择正确的损失函数

通常无法将指标直接传递给损失函数,损失函数需要的

- 小批量数据能计算出来
- 必须是可微的(反向传播需要)

一个例子是分类任务的常见的指标 ROC AUC(后面接触到再详细解释),就不能直接传递给损失函数,因此一般会对代理的指标进行优化,例如交叉熵,交叉熵越低,ROC AUC 越高.

下面是常见的问题最后一层激活函数和损失函数的对应关系

|  问题类型  | 最后一层激活函数  | 损失函数|
|  ----  | ----  |---|
| 二分类  | `sigmoid` |`binary_crossentropy`|
| 多分类单标签  | `softmax` |`categorical_crossentropy`|
|多分类多标签|`sigmoid`|`binary_crossentropy`|
|回归问题|None|`mse`|


## 构建较大的过拟合模型

如果顺利,现在我们手头已经有了一个较小的模型,足够打败基线.

现在问题来了,这个模型是否足以归纳输入的数据?模型的容量够不够?

前面的例子我们见过,如果只有一个全连接层的模型,可以在 mnist 数据上工作,当不足以解决问题.还是那句话,机器学习是平衡的艺术,我们需要模型在欠拟合/过拟合之间平衡.模型的学习能力在能力不足/能力过剩之间平衡.为了准确知道边界,我们必须首先跨国边界才行.

为了搞清楚需要的模型容量,我们需要构建一个更大的过拟合模型.具体流程上一章我们已经接触过了.

- 增加更多的层
- 扩大每层的特征数量
- 每个轮次训练更多样本

要始终监控训练损失和验证损失等需要关心的指标,当模型开始过拟合时,要明确知道我们已经跨过了边界.
